In [ ]:
%cd /data/codes/apa/serve/
from src.utils.arpa import arpa_to_ipa
from multiprocessing.pool import Pool
import numpy as np
import requests
import librosa
import json
import ray

In [ ]:
wav_path = "/data/codes/apa/wav/She's paying for some jewelry at a checkout.wav"
transcript = "She's paying for some jewelry at a check out".upper()

waveform, sr = librosa.load(wav_path, sr=16000)

params = {
    "audio": waveform.tolist(),
    "transcript": transcript
}

In [ ]:
inputs = [params.copy() for i in range(1)]

@ray.remote
def send_query(params):
    response = requests.post("http://14.162.145.55:9999/scoring", json=params)
    return response

outputs = ray.get([send_query.remote(params) for params in inputs])
outputs = [json.loads(output.content) for output in outputs]
